In [1]:
import numpy as np
import pandas as pd
import sqlite3
import os

In [2]:
# Number of advertisments in the database
def get_number_ads():
    db_connector = sqlite3.connect('./scraping_bienici.db')
    cursor = db_connector.cursor()
    cursor.execute("""SELECT COUNT(*) FROM real_estate_ad""")
    result = cursor.fetchall()[0][0]
    db_connector.close()
    return result

number_ads = get_number_ads()
print("%s advertisments have been scraped" %number_ads)

7014 advertisments have been scraped


In [3]:
# Export to csv
def get_col_names():
    conn = sqlite3.connect("./scraping_bienici.db")
    c = conn.cursor()
    c.execute("select * from real_estate_ad")
    descr = c.description
    conn.close()
    return [member[0] for member in descr]
    
def get_data_as_dataframe():
    db_connector = sqlite3.connect('./scraping_bienici.db')
    cursor = db_connector.cursor()
    cursor.execute("""select * FROM real_estate_ad""")
    result = cursor.fetchall()
    
    col_names = get_col_names()
    
    df = pd.DataFrame(list(result))
    df.columns = col_names
    
    db_connector.close()
    return df

def export_db_to_csv(dataFrame):
    dataFrame.to_csv('./scraping_bienici.csv', sep=',', index=False)

In [4]:
df = get_data_as_dataframe()
export_db_to_csv(df)

In [5]:
df.head(4)

,url,scraping_date,title,price,charges,place,surface,construction_year,description,number_pictures,...,lift,contact,contact_address,contact_rcs,contact_type,ref_annonce,publication_date,modification_date,infos_quartier,other_info
0,https://www.bienici.com/annonce/location//appa...,2016-10-09@00:30,Appartement 2 pièces 36 m²,1150,70,- Grandes Carrières - Clichy,36,,Description HTML_format -- Haut Rue Damr&eacut...,6,...,,Acopa Damrémont,19 Rue Damrémont - 75018 Paris,RCS : 512097031,Agence,3987,5 oct. 2016,,"<span><a href=""/decouvrez/paris/paris-18e/gran...",Exposé Est \n\&\\n1 WC\n\&\\n
1,https://www.bienici.com/annonce/location//appa...,2016-10-09@00:30,Appartement 2 pièces 36 m²,1150,70,- Grandes Carrières - Clichy,36,,Description HTML_format -- Haut Rue Damr&eacut...,6,...,,Acopa Damrémont,19 Rue Damrémont - 75018 Paris,RCS : 512097031,Agence,3989,6 oct. 2016,,"<span><a href=""/decouvrez/paris/paris-18e/gran...",Exposé Est \n\&\\n1 WC\n\&\\n
2,https://www.bienici.com/annonce/location//appa...,2016-10-09@00:30,Appartement 3 pièces 85 m²,2165,165,- Jules Joffrin,85,,Description HTML_format -- Appartement 3/4 pi&...,11,...,,Acopa Immobilière Marcadet,77 Bd Barbes - 75018 Paris,RCS : 70305443384,Agence,3956,16 sept. 2016,,"<span><a href=""/decouvrez/paris/paris-18e/jule...",
3,https://www.bienici.com/annonce/location/paris...,2016-10-09@00:30,Studio 31 m²,930,105,Paris 10e 75010 - Grange aux Belles - Terrage,31,,"République - Studio de 31 m². M° République, à...",5,...,,CONSULT IM - CONSULT'IM,"17, Rue De Chateaudun - 75009 Paris",RCS : 511411324,Agence,10185379,7 oct. 2016,,"<span><a href=""/decouvrez/paris/paris-10e/gran...",


In [6]:
# Get the size of the database
size_db = os.path.getsize("./scraping_bienici.db") # in bytes
print("The size of the base of sales is %s MBytes" %(size_db/10**6))

The size of the base of sales is 15.11424 MBytes


In [7]:
# Proportion of ads where charge is known
def get_proportion_charges():
    db_connector = sqlite3.connect('./scraping_bienici.db')
    cursor = db_connector.cursor()
    cursor.execute("""SELECT COUNT(*)
                      FROM real_estate_ad
                      WHERE charges != ''""")
    result = float(cursor.fetchall()[0][0])
    db_connector.close()
    return round(result/get_number_ads()*100, 1)

In [8]:
print("Charges are knows in %s%% of the real estate advertisments" %get_proportion_charges())

Charges are knows in 79.1% of the real estate advertisments


In [9]:
# Proportion of ads where contruction year is known
def get_proportion_consruction_year():
    db_connector = sqlite3.connect('./scraping_bienici.db')
    cursor = db_connector.cursor()
    cursor.execute("""SELECT COUNT(*)
                      FROM real_estate_ad
                      WHERE construction_year != ''""")
    result = float(cursor.fetchall()[0][0])
    db_connector.close()
    return round(result/get_number_ads()*100, 1)

In [10]:
print("The construction year is knows in %s%% of the real estate advertisments" %get_proportion_consruction_year())

The construction year is knows in 36.9% of the real estate advertisments


In [11]:
def get_nb_ads_max_rooms(n_max_rooms):
    db_connector = sqlite3.connect('./scraping_bienici.db')
    cursor = db_connector.cursor()
    args = [n_max_rooms]
    cursor.execute("""SELECT COUNT(*)
                      FROM real_estate_ad
                      WHERE charges != '' and number_rooms <= ?""", args)
    result = int(cursor.fetchall()[0][0])
    db_connector.close()
    print("%s flats out of %s have at most %s rooms." %(result, get_number_ads(), n_max_rooms))
    return result

In [12]:
n = get_nb_ads_max_rooms(4)

4760 flats out of 6848 have at most 4 rooms.
